<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Web Scraping

_Author: Dave Yerrington (SF)_

---

## This notebook requires Python 2 to run Selenium.

### Method 1: 

Launch Anaconda
Switch Applications On from root to Python27
Launch Jupyter Notebook
Open Python 2 notebook


### Method 2: Run Anaconda in a Python 2 Container

#### Run bash from the shell command line (if not already in bash)
$ bash

#### Enter Python 2 container
bash-3.2$ source activate python27

#### Verify Python Version ####
(python27) bash-3.2$ python --version
Python 2.7.13 :: Continuum Analytics, Inc.

#### Launch Anaconda from the command line in the container
(Python27) bash-3.2$ /anaconda/envs/Python27/bin/jupyter_mac.command ; exit;

or

(Python27) bash-3.2$ /anaconda/envs/Python27/bin/anaconda-navigator


In [2]:
import platform
platform.python_version()

'2.7.13'

In [3]:
import sys
print sys.version

2.7.13 |Continuum Analytics, Inc.| (default, Dec 20 2016, 23:05:08) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]


In [4]:
# test import
from selenium import webdriver

## Before Class

#### Install Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

To install Selenium, use one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`


#### Install GeckoDriver

You will also need GeckoDriver (this assumes you are using Homebrew for Mac): 

- ```brew install geckodriver```

#### Install Firefox

Additionally, you will need to have downloaded the [Firefox browser](https://www.mozilla.org/en-US/firefox/new/?utm_source=google&utm_medium=cpc&utm_campaign=Firefox-Brand-US-GGL-Exact&utm_term=firefox&utm_content=A144_A203_A006336&gclid=Cj0KEQjwnPLKBRC-j7nt1b7OlZwBEiQAv8lMLJUyReT6cPzSYdmEA6uD3YDoieuuuusddgAU7XH6smEaAoje8P8HAQ&gclsrc=aw.ds) for the application in this lesson.

## Learning Objectives
- Revisit how to locate elements on a webpage
- Aquire unstructure data from the internet using Beautiful soup.
- Discuss limitations associated with simple requests and urllib libraries
- Introduce Selenium as a solution, and implement a scraper using selenium

## Lesson Guide

- [Introduction](#intro)
- [Building a web scraper](#building-scraper)
- [Retrieving data from the HTML page](#retrieving-data)
    - [Retrieving the restaurant names](#retrieving-names)
    - [Challenge: Retrieving the restaurant locations](#retrieving-locations)
    - [Retrieving the restaurant prices](#retrieving-prices)
    - [Retrieving the restaurant number of bookings](#retrieving-bookings)


- [Introducting Selenium](#selenium)
    - [Running JavaScript before scraping](#selenium-js)
    - [Using regex to only get digits](#selenium-regex)
    - [Challenge: Use Pandas to create a DataFrame of bookings](#challenge-pandas)
    - [Auto-typing using Selenium](#selenium-typing)


- [Summary](#summary)

<a id="intro"></a>
## Introduction

In this codealong lesson, we'll build a web scraper using urllib and BeautifulSoup. We will also explore how to use a headless browser called Selenium.

We'll begin by scraping OpenTable's DC listings. We're interested in knowing the restaurant's **name, location, price, and how many people booked it today.**

OpenTable provides all of this information on this given page: http://www.opentable.com/washington-dc-restaurant-listings

Let's inspect the elements of this page to assure we can find each of the bits of information in which we're interested.

---

<a id="building-scraper"></a>
## Building a web scraper

Now, let's build a web scraper for OpenTable using urllib and Beautiful Soup:

In [13]:
# import our necessary first packages
from bs4 import BeautifulSoup
import urllib

In [35]:
# set the url we want to visit
url = "http://www.opentable.com/washington-dc-restaurant-listings"

# visit that url, and grab the html of said page
html = urllib.urlopen(url).read()

At this point, what is in html (main level in DOM - Document Object Model)?

In [36]:
html[:500]   # first 500 characters

'          <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/> <title>Washington, D.C. Area Restaurants List | OpenTable</title>  <meta  name="description" content="Find Washington, D.C. Area restaurants. Search by location, cuisine, or price to refine restaurant results in the Washington, D.C. Area area." > </meta>  <meta  name="robots" content="noindex" > </meta>    <link rel="shortcut icon" href="//components.otst'

We will need to convert this html objct into a soup object so we can parse it using python and BS4

In [37]:
# convert this into a soup object
soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
# soup   # entire html content

<a id="retrieving-data"></a>
### Retrieving data from the HTML page

Let's first find each restaurant name listed on the page we've loaded. How do we find the page location of the restaurant? (Hint: We need to know where in the **HTML** the restaurant element is housed.) In order to find the HTML that renders the restaurant location, we can use Google Chrome's Inspect tool:

> http://www.opentable.com/washington-dc-restaurant-listings

> 1. Visit the URL above. 

> 2. Right-click on an element you are interested in, then choose Inspect (in Chrome). 

> 3. This will open the Developer Tools and show the HTML used to render the selected page element. 

> Throughout this lesson, we will use this method to find tags associated with elements of the page we want to scrape.

See if you can find the restaurant name on the page. Keep in mind there are many restaurants loaded on the page.

In [44]:
# print the restaurant names
soup.find_all(name='span', attrs={'class':'rest-row-name-text'})

[<span class="rest-row-name-text">Ruffino's - Arlington</span>,
 <span class="rest-row-name-text">Joe's Place Pizza and Pasta</span>,
 <span class="rest-row-name-text">Founding Farmers - DC</span>,
 <span class="rest-row-name-text">Rasika West End</span>,
 <span class="rest-row-name-text">Blue Duck Tavern</span>,
 <span class="rest-row-name-text">Farmers Fishers Bakers</span>,
 <span class="rest-row-name-text">Founding Farmers - Tysons</span>,
 <span class="rest-row-name-text">Filomena Ristorante</span>,
 <span class="rest-row-name-text">District Commons</span>,
 <span class="rest-row-name-text">Ambar - Arlington</span>,
 <span class="rest-row-name-text">Eddie V's - Tysons Corner</span>,
 <span class="rest-row-name-text">Clyde's of Georgetown</span>,
 <span class="rest-row-name-text">BlackSalt</span>,
 <span class="rest-row-name-text">Chez Billy Sud</span>,
 <span class="rest-row-name-text">Seasons 52 - Tysons Corner</span>,
 <span class="rest-row-name-text">Fogo de Chao Brazilian Stea

It is important to always keep in mind the data types that were returned. Note this is a `list`, and we know that immediately by observing the outer square brackets and commas separating each tag.

Next, note the elements of the list are `Tag` objects, not strings. (If they were strings, they would be surrounded by quotes.) The Beautiful Soup authors chose to display a `Tag` object visually as a text representation of the tag and its contents. However, being an object, it has many methods that we can call on it. For example, next we will use the `encode_contents()` method to return the tag's contents encoded as a Python string.

<a id="retrieving-names"></a>
#### Retrieving the restaurant names

Now that we found a list of tags containing the restaurant names, let's think how we can loop through them all one-by-one. In the following cell, we'll print out the name (and **only** the clean name, not the rest of the html) of each restaurant.

In [45]:
# for each element you find, print out the restaurant name
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    print entry.encode_contents()

Ruffino's - Arlington
Joe's Place Pizza and Pasta
Founding Farmers - DC
Rasika West End
Blue Duck Tavern
Farmers Fishers Bakers
Founding Farmers - Tysons
Filomena Ristorante
District Commons
Ambar - Arlington
Eddie V's - Tysons Corner
Clyde's of Georgetown
BlackSalt
Chez Billy Sud
Seasons 52 - Tysons Corner
Fogo de Chao Brazilian Steakhouse - Tyson's
Sugar Factory - Pentagon Mall
CIRCA at Foggy Bottom
Il Canale
Barrel &amp; Bushel
2941 Restaurant
Nostos Restaurant
Et Voila
Tupelo Honey - Arlington
Chef Geoff's Tysons
Matchbox - Pentagon City
Texas Jacks Barbecue
The Capital Grille - Tysons Corner
Earls Kitchen + Bar - Tysons Corner
J. Gilbert’s – Wood Fired Steaks &amp; Seafood - McLean
Cafe Deluxe - Tysons Corner
Palette 22
Clyde's of Chevy Chase
Barley Mac
Chima Steakhouse - Washington
CIRCA at Dupont
Tony &amp; Joe's Seafood Place
CIRCA at Clarendon
Sequoia
Sette Osteria
Chef Geoff's (New Mexico Ave.)
The Palm Washington DC
Peacock Café
La Chaumiere
Grillfish DC
LIA'S
Maggiano's - C

Great!

<a id="retrieving-locations"></a>
#### Challenge: Retrieving the restaurant locations

Can you repeat that process for finding the location? For example, barmini by Jose Andres is in the location listed as "Penn Quarter" in our search results.

In [46]:
# first, see if you can identify the location for all elements -- print it out
# <span class="rest-row-meta--location rest-row-meta-text">Arlington</span>
soup.find_all(name='span', attrs={'class':'rest-row-meta--location rest-row-meta-text'})

[<span class="rest-row-meta--location rest-row-meta-text">Arlington</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Arlington</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Foggy Bottom</span>,
 <span class="rest-row-meta--location rest-row-meta-text">West End</span>,
 <span class="rest-row-meta--location rest-row-meta-text">West End</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Georgetown</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Tysons Corner / McLean</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Georgetown</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Foggy Bottom</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Arlington</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Tysons Corner / McLean</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Georgetown</span>,
 <span class="rest-row-meta--location rest-row-meta-tex

In [47]:
# now print out EACH location for the restaurants
for entry in soup.find_all(name='span', attrs={'class':'rest-row-meta--location rest-row-meta-text'}):
    print entry.encode_contents()

Arlington
Arlington
Foggy Bottom
West End
West End
Georgetown
Tysons Corner / McLean
Georgetown
Foggy Bottom
Arlington
Tysons Corner / McLean
Georgetown
Palisades Northwest
Georgetown
Tysons Corner / McLean
Tysons Corner / McLean
Arlington
Foggy Bottom
Georgetown
Tysons Corner / McLean
Falls Church
Vienna
Palisades Northwest
Arlington
Tysons Corner / McLean
Arlington
Arlington
Tysons Corner / McLean
Tysons Corner / McLean
McLean
Tysons Corner / McLean
Arlington
Friendship Heights
Arlington
Vienna
Dupont Circle
Georgetown
Arlington
Georgetown
Dupont Circle
Georgetown
Dupont Circle
Georgetown
Georgetown
Dupont Circle
Bethesda / Chevy Chase
Friendship Heights
Tysons Corner / McLean
Chevy Chase
Tysons Corner / McLean
Georgetown
Arlington
Dupont Circle
Georgetown
Georgetown
Bethesda / Chevy Chase
Downtown
Arlington
Dupont Circle
Foggy Bottom
Georgetown
Georgetown
Arlington
Arlington
Falls Church
Arlington
Alexandria
Georgetown
Foggy Bottom
Arlington
Dupont Circle
West End
Georgetown
Georget

<a id="retrieving-prices"></a>
#### Retrieving the restaurant prices

Ok, we've figured out the restaurant name and location. Now we need to grab the price (number of dollar signs on a scale of one to four) for each restaurant. We'll follow the same process.

In [48]:
# print out all prices
soup.find_all('div', {'class':'rest-row-pricing'})

[<div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pri

In [49]:
# print out EACH number of dollar signs per restaurant
# this one is trickier to eliminate the html. Hint: try a nested find
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    print entry.find('i').encode_contents()

  $    $      
  $    $      
  $    $      
  $    $    $    
  $    $    $    
  $    $      
  $    $      
  $    $    $    
  $    $      
  $    $      
  $    $    $    
  $    $      
  $    $    $    
  $    $      
  $    $      
  $    $    $    $  
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $      
  $    $    $    
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $    $    
  $    $    $    
  $    $    $    
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $    $    
  $    $      
  $    $    $    
  $    $      
  $    $      
  $    $  

That looks great, but what if I wanted just the number of dollar signs per restaurant? Can you figure out a way to simply print out the number of dollar signs per restaurant listed?

In [50]:
# print the number of dollars signs per restaurant
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    price = entry.find('i').renderContents()
    print price.count('$')

2
2
2
3
3
2
2
3
2
2
3
2
3
2
2
4
2
2
2
2
2
2
2
2
2
2
2
4
2
3
2
2
2
2
4
2
2
2
2
2
2
4
2
2
2
2
2
4
4
3
3
3
2
2
2
4
3
2
3
2
2
4
2
3
2
2
2
2
4
2
2
3
3
3
2
4
2
2
2
4
3
2
2
2
2
3
3
3
4
2
2
2
2
2
2
2
2
2
2
2


Phew, nice work. 

<a id="retrieving-bookings"></a>
#### Retrieving the restaurant number of bookings

One more, right? We only need to find the number times a restaurant was booked. In the next cell, print out all objects that contain the number of times the restaurant was booked.

In [51]:
# print out all objects that contain the number of times the restaurant was booked
# fails because of antibot delay
soup.find_all('div', {'class':'booking'})

[]

That's weird -- an empty set. Did we find the wrong element? What's going on here? Discuss.

How can we debug this? Any ideas?

In [52]:
# let's first try printing out all 'div' objects
#  NOTE: This is a too many objects to store in this notebook!
#        So, uncomment the code below to run it.

# for entry in soup.find_all('div'):
#     print entry

I still don't see it. Let's search our entire soup object:

In [31]:
# print out soup, do command+f for "booked ". 
#   Uncomment the below to run.

# soup

What do you notice? Why is this happening?
A: Need Selenium to overcome the anitbot feature

<a id="selenium"></a>
## Introducing Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

To install Selenium, use one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`

You will also need GeckoDriver (this assumes you are using Homebrew for Mac): 

- ```brew install geckodriver```

Additionally, you will need to have downloaded the [Firefox browser](https://www.mozilla.org/en-US/firefox/new/?utm_source=google&utm_medium=cpc&utm_campaign=Firefox-Brand-US-GGL-Exact&utm_term=firefox&utm_content=A144_A203_A006336&gclid=Cj0KEQjwnPLKBRC-j7nt1b7OlZwBEiQAv8lMLJUyReT6cPzSYdmEA6uD3YDoieuuuusddgAU7XH6smEaAoje8P8HAQ&gclsrc=aw.ds) for the application in this lesson.

In [53]:
# needs Python 2
!pip install selenium

In [54]:
# import (only works in Python 3)
from selenium import webdriver

Selenium requires us to determine a default browser to run. I'm going to opt for Firefox, but Chromium is also a very common choice. http://selenium-python.readthedocs.io/faq.html

In [35]:
# STOP
# what is going to happen when I run the next cell?

In [ ]:
# create a driver called Firefox
driver = webdriver.Firefox()

Pretty crazy, right? Let's close that driver.

In [ ]:
# close it
driver.close()

In [58]:
# let's boot it up, and visit a URL of our choice
driver = webdriver.Firefox()
driver.get("http://www.python.org")

Awesome. Now we're getting somewhere: programmatically controlling our browser like a human.

Let's return to our problem at hand. We need to visit the OpenTable listing for DC. Once there, we need to get the html to load. In the next cell, prove you can programmatically visit the page.

In [59]:
# visit our OpenTable page
driver = webdriver.Firefox()
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")
# always good to check we've got the page we think we do
assert "OpenTable" in driver.title

In [60]:
driver.title

u'Washington, D.C. Area Restaurants List | OpenTable'

In [61]:
driver.close()

<a id="selenium-js"></a>
### Running JavaScript before scraping

Now, to resolve our JavaScript problem, there's a few things we can do. What I'll do in this case is request that the page load, wait one second, and then I'm going to grab the source html from the page. Because the page should believe I'm visiting from a live connection on a browser client, the JavaScript should render to be a part of the page source. I can then grab the page source.

In [62]:
# import sleep
from time import sleep

In [63]:
# visit our relevant page
driver = webdriver.Firefox()
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")
# wait one second
sleep(1)
#grab the page source
html = driver.page_source

**Pop Quiz:** What do we need to do with this HTML?

In [ ]:
# BeautifulSoup it!
html = BeautifulSoup(html, "lxml")

Now, let's return to our earlier problem: How do we locate bookings on the page?

In [66]:
# print out the number bookings for all restaurants
html.find_all('div', {'class':'booking'})

[<div class="booking"><span class="tadpole"></span>Booked 455 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 132 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 132 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 305 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 307 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 152 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 92 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 86 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 54 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 71 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 52 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 55 times today</div>,
 <div class="booking"><span class=

In [67]:
# now print out each booking for the listings using a loop
for entry in html.find_all('div', {'class':'booking'}):
    print entry

<div class="booking"><span class="tadpole"></span>Booked 455 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 132 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 132 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 305 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 307 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 152 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 92 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 86 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 54 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 71 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 52 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 55 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 5

Let's grab just the text of each of these entries.

In [68]:
# do the same as above, but grabbing only the text content
for entry in html.find_all('div', {'class':'booking'}):
    print entry.text

Booked 455 times today
Booked 132 times today
Booked 132 times today
Booked 305 times today
Booked 307 times today
Booked 152 times today
Booked 92 times today
Booked 86 times today
Booked 54 times today
Booked 71 times today
Booked 52 times today
Booked 55 times today
Booked 56 times today
Booked 38 times today
Booked 117 times today
Booked 63 times today
Booked 68 times today
Booked 43 times today
Booked 41 times today
Booked 46 times today
Booked 45 times today
Booked 71 times today
Booked 35 times today
Booked 33 times today
Booked 31 times today
Booked 31 times today
Booked 48 times today
Booked 79 times today
Booked 29 times today
Booked 19 times today
Booked 59 times today
Booked 26 times today
Booked 53 times today
Booked 29 times today
Booked 20 times today
Booked 19 times today
Booked 80 times today
Booked 34 times today
Booked 29 times today
Booked 47 times today
Booked 21 times today
Booked 43 times today
Booked 37 times today
Booked 15 times today
Booked 24 times today
Boo

In [69]:
driver.close()

We've succeeded!

<a id="selenium-regex"></a>
### Using regex to only get digits

But we can clean this up a little bit. We're going to use regular expressions (regex) to grab only the digits that are available in each of the text.

The best way to get good at regex is to, well, just keep trying and testing: http://pythex.org/

In [71]:
# import regex
import re

Given we haven't covered regex, I'll show you how to use the search function to match any given digit.

In [72]:
# for each entry, grab the text
for booking in html.find_all('div', {'class':'booking'}):
    # match all digits
    match = re.search(r'\d+', booking.text)
    
    if match:
        # print if found
        print match.group()
    else:
        # otherwise pass
        pass

455
132
132
305
307
152
92
86
54
71
52
55
56
38
117
63
68
43
41
46
45
71
35
33
31
31
48
79
29
19
59
26
53
29
20
19
80
34
29
47
21
43
37
15
24
27
37
24
21
31
24
22
20
36
48
14
38
40
52
49
52
54
5
17
32
12
21
13
19
34
17
27
14
18
37
21
28
63
53
43
49
25
10
14
31
25
22
12
30
25
16
25
18
19
18
17
15
6


Before we demonstrate all the other amazing things about headless browsers, let's finish up collecting the data we want from this current example. Do you suppose the html parsing we wrote above will still work on the page source we've grabbed from our headless browser?

To be most efficient, we want to only do a single loop for each entry on the page. That means we want to find what element all of other other elements (name, location, price, bookings) is housed within. Where on the page is each entry located?

In [73]:
# print out all entries
#   NOTE: Has many entries. Uncomment the below code to run it!

soup.find_all('div', {'class':'result content-section-list-row cf with-times'})

[<div class="result content-section-list-row cf with-times" data-id="0" data-index="1" data-lat="38.8974400" data-lon="-77.1242130" data-offers="" data-rid="66715"><div class="rest-row with-image"> <div class="rest-row-image"> <a href="/ruffinos-arlington" onclick="OT.BestAnalytics.logRestaurantVisit(66715)" target="_blank"><img alt="photo of ruffino's - arlington restaurant" class="lazy rest-image" data-src="//resizer.otstatic.com/v2/profiles/legacy/66715.jpg" src="//media.otstatic.com/search-result-node/images/no-image.png"/></a></div> <div class="rest-row-info"><div class="rest-row-header"> <a class="rest-row-name rest-name " href="/ruffinos-arlington" onclick="OT.BestAnalytics.logRestaurantVisit(66715)" target="_blank"> <span class="rest-row-name-text">Ruffino's - Arlington</span> </a> </div> <div class=" flex-row-justify"> <div class="rest-row-review"> <div class="star-rating review-container"><div class="star-wrapper small"><div class="all-stars"></div><div class="all-stars fille

Look over the page. Does every single entry have each element we're seeking?
> I did this previously. I know for a fact that not every element has a number of recent bookings. That's probably exactly why OpenTable houses this in JavaScript: they want to continously update the number of bookings with the most relevant number of values.

In [74]:
# What happens when a booking is not available?
# Print out each booking entry, using the identification code we wrote above

for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    print entry.find('div', {'class':'booking'})

None
None
<div class="booking"><span class="tadpole"></span>Booked 455 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 132 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 132 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 305 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 307 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 152 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 92 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 86 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 54 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 71 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 52 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 55 times today</div>
<div class="booking"><span class="tadpole"></spa

In [75]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
        print entry.find('div', {'class':'booking'}).text

AttributeError: 'NoneType' object has no attribute 'text'

What do you notice takes the place when booking is not found?

We could use exception handling (`try`/`except` blocks) to resolve this. However, exceptions should only be used to handle rare or unexpected errors -- never for normal program flow.

In this case, we expect that some entries will be zero. So, we can just use an `if` statement that tests whether there are any `divs` present; if not, display `'ZERO'`. Here's a demo:

In [76]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    booking_tag = entry.find('div', {'class':'booking'})
    
    if booking_tag:
        print booking_tag.text
    else:
        print 'ZERO'

ZERO
ZERO
Booked 455 times today
Booked 132 times today
Booked 132 times today
Booked 305 times today
Booked 307 times today
Booked 152 times today
Booked 92 times today
Booked 86 times today
Booked 54 times today
Booked 71 times today
Booked 52 times today
Booked 55 times today
Booked 56 times today
Booked 38 times today
Booked 117 times today
Booked 63 times today
Booked 68 times today
Booked 43 times today
Booked 41 times today
Booked 46 times today
Booked 45 times today
Booked 71 times today
Booked 35 times today
Booked 33 times today
Booked 31 times today
Booked 31 times today
Booked 48 times today
Booked 79 times today
Booked 29 times today
Booked 19 times today
Booked 59 times today
Booked 26 times today
Booked 53 times today
Booked 29 times today
Booked 20 times today
Booked 19 times today
Booked 80 times today
Booked 34 times today
Booked 29 times today
Booked 47 times today
Booked 21 times today
Booked 43 times today
Booked 37 times today
Booked 15 times today
Booked 24 times

After previously completing this, we observed that all other elements WILL be returned. This means we do not have to always handle these cases.

<a id="challenge-pandas"></a>
### Challenge: Use Pandas to create a DataFrame of bookings

However, the onus is on you to now put all the pieces together.

Loop through each entry. For each entry, grab the relevant information we want (name, location, price, bookings). Produce a dataframe with the columns "name","location","price","bookings" that contains the 100 entries we would like.

In [77]:
import pandas as pd

In [78]:
# I'm going to create my empty df first
dc_eats = pd.DataFrame(columns=["name","location","price","bookings"])

In [75]:
# Put code here that populates the DataFrame using Selenium and BeautifulSoup!

In [86]:
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    # grab the name
    name =  entry.find('span', {'class':'rest-row-name-text'}).text
    
    # grab the location 
    location = entry.find('span', {'class':'rest-row-meta--location rest-row-meta-text'}).renderContents()
    
    # grab the price
    price =  entry.find('div', {'class':'rest-row-pricing'}).find('i').renderContents().count('$')
    
    # try to find the number of bookings
    bookings = 'NA'
    booking_tag = entry.find('div', {'class':'booking'})
    if booking_tag:
        match = re.search(r'\d+', booking_tag.text)
        
        if match:
            bookings = match.group()
    
    dc_eats.loc[len(dc_eats)]=[name, location, price, bookings]

In [87]:
# check out our work
dc_eats.head()

,name,location,price,bookings
0,Ruffino's - Arlington,Arlington,2,NA
1,Joe's Place Pizza and Pasta,Arlington,2,NA
2,Founding Farmers - DC,Foggy Bottom,2,455
3,Rasika West End,West End,3,132
4,Blue Duck Tavern,West End,3,132


Awesome! We succeeded.

<a id="selenium-typing"></a>
### Auto-typing using Selenium

Now, let's explore some of the other functionality of a webdriver. We've barely scratched the surface.

In [80]:
# we can send keys as well
# import
from selenium.webdriver.common.keys import Keys

In [81]:
# open Firefox
driver = webdriver.Firefox()
# visit Python
driver.get("http://www.python.org")
# verify we're in the right place
assert "Python" in driver.title

Let's try automatedly typing `pycon` in the search box and hitting the return key:

In [82]:
# find the search position
elem = driver.find_element_by_name("q")

# clear it
elem.clear()

# type in pycon
elem.send_keys("pycon")

# send those keys
elem.send_keys(Keys.RETURN)

In [83]:
# close
driver.close()

In [84]:
# all at once:
driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title

elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
driver.close()

The above example (and many others) are available in the Selenium docs: http://selenium-python.readthedocs.io/getting-started.html

What is especially important is exploring functionality like locating elements: http://selenium-python.readthedocs.io/locating-elements.html#locating-elements

FAQ:
http://selenium-python.readthedocs.io/faq.html

### Summary

In this lesson, we used the Beautiful Soup library to locate elements on a website then scrape their text. We also used the Selenium headless browser to run JavaScript first before retrieving the page contents.